In [1]:
# imports

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

### Step 1: Establish broad API call
* ensures API ok


In [1]:
import json

import requests
import os

FS_API_KEY = os.getenv('FOURSQUARE_API_KEY')


print(FS_API_KEY)

location = "Vancouver, Canada"
url = "https://api.foursquare.com/v3/places/search?near=" + location

# Create dictionary for headers

payload = {}
headers = {
        "Accept": "application/json",
        'Authorization': FS_API_KEY
            }

response = requests.request("GET", url, headers=headers, data=payload)

print(response.json())

fsq3IfwEJ3A+/5rg/MDnHEO7RgiDAxoRIW9j7zW/C0ScVlE=
{'results': [{'fsq_id': '4bdf6cddffdec9287a09eca1', 'categories': [{'id': 16039, 'name': 'Urban Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}}], 'chains': [], 'distance': 998, 'geocodes': {'main': {'latitude': 49.241325, 'longitude': -123.11111}, 'roof': {'latitude': 49.236986, 'longitude': -123.110884}}, 'link': '/v3/places/4bdf6cddffdec9287a09eca1', 'location': {'address': '4600 Cambie St', 'country': 'CA', 'formatted_address': '4600 Cambie St, Vancouver BC V5Z 2Z1', 'locality': 'Vancouver', 'postcode': 'V5Z 2Z1', 'region': 'BC'}, 'name': 'Queen Elizabeth Park', 'related_places': {'children': [{'fsq_id': '4e07cfdfb61c60b04547fd16', 'name': 'Queen Elizabeth Parking Lot'}, {'fsq_id': '4e5957dd52b1b78b819bf083', 'name': 'Celebration Pavilion'}, {'fsq_id': '637ab2e6da100869764d3264', 'name': 'Quarry Gardens'}, {'fsq_id': '4aac6667f964a520d85d20e3', 'name': 'Bloedel Floral Conserv

## Step 2: Prepare and Test Lat Long info from SQL
* ensure syntax is correct
* update Postgres DB to avoid potential concatenation complication in code.
* 

In [3]:
## SQL to create a new column in the City_bike Table. 
# Performed in Postgres

'''UPDATE city_bikes
SET lat_long = (select concat(latitude,',',longitude));'''



"UPDATE city_bikes\nSET lat_long = (select concat(latitude,',',longitude));"

## Step 3: Create DataFrame of Lat/Long and iterate to create API calls

In [103]:
import pandas as pd

data = pd.read_csv('../data/city_bikes.csv')
vancouver_stations = pd.DataFrame(data)
vancouver_stations_TEST= vancouver_stations.head(5)
vancouver_stations_TEST


,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra_ebikes,extra_has_ebikes,extra_last_updated,lat_long
0,5,29,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie,2023-09-21 19:03:51.469000,6,True,1695322925,"49.262487,-123.114397"
1,12,4,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station,2023-09-21 19:03:51.456000,0,True,1695322945,"49.274566,-123.121817"
2,13,13,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty,2023-09-21 19:03:51.457000,1,True,1695322662,"49.279764,-123.110154"
3,11,5,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall),2023-09-21 19:03:51.458000,4,True,1695322780,"49.260599,-123.113504"
4,9,7,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash,2023-09-21 19:03:51.477000,4,True,1695322949,"49.264215,-123.117772"


## Foursquare API and Dataframe
### Code Overview (foursquare) 
* function takes lat and long and radius as arguments. 
* GET parameters include desired categories, and field information
* API response (json) file is iterated to retrieve data (name, popularity, price etc)
* data (dictionary) is appended to a list callled "FS_API_result"
* List of lat/longs from CityBikes (vancouver_stations) is itererated.
* Each lat/log in Vancouver_stations creates a new API call in the 'foursquare' function. 
* Results from each call (FS_API_Result) are in turn iterated and appended to the "All Results" list. 
* All_Results is a tidy dictionary that does not require json normalization. 
* DataFrame is created using All_Results dictionary. 
* DataFrame is dumped to json in ../data folder for future reference. 


In [112]:
import requests
import os

FS_API_KEY = os.getenv('FOURSQUARE_API_KEY')
def foursquare(latitude, longitude,station_id, radius):
    ## Initialize an empty list to store the results
    FS_API_Result = []
    
    url = 'https://api.foursquare.com/v3/places/search'
    category_filter = "11062,13003,13004,13005,13006,13007,13008,13009,13010,13011,13012,13013,13014,13015,13016,13017,13018,13019,13020,13021,13022,13023,13024,13025,13059"
    params = { "ll": f"{latitude},{longitude}", "radius": radius, "categories": {category_filter},'fields': 'name,popularity,rating,price,distance,fsq_id'}
    headers = { "Accept": "application/json", "Authorization": FS_API_KEY }
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code != 200:
        return f"Error: {response.status_code}"
    else:
        data = response.json()
        for i in data['results']:
            details = {
            'name': i.get('name', None),
            'popularity': i.get('popularity', None),
            'rating': i.get('rating', None),
            'price': i.get('price', None),
            'distance': i.get('distance', None),
            'fsq_id': i.get('fsq_id', None)
        }
            FS_API_Result.append(details)
        return FS_API_Result

all_results = []  # Initialize empty list to accumulate results from each API call.
    ## Loop latitude,longitude from station to get details
for index, row in vancouver_stations.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    station_id = row['id']
    result_list = foursquare(latitude, longitude, station_id, 1000)

    for result in result_list:
        # Create a new dictionary with latitude and longitude, id of station, then update it with the result dictionary
        combined_result = {'latitude': latitude, 'longitude': longitude, 'station_id': station_id}
        combined_result.update(result)
        
    ## Append the combined dictionary to 'all_results'
    all_results.append(combined_result)
print(all_results)

[{'latitude': 49.262487, 'longitude': -123.114397, 'station_id': '7a19c49f486d7c0c02b3685d7b240448', 'name': 'Mahony & Sons', 'popularity': 0.9952687543650325, 'rating': 7.4, 'price': None, 'distance': 811, 'fsq_id': '53eee0b9498e381a6c7eaa98'}, {'latitude': 49.274566, 'longitude': -123.121817, 'station_id': '32603a87cfca71d0f7dfa3513bad69d5', 'name': 'Localcoin Bitcoin ATM - Red Card Sports Bar + Eatery', 'popularity': 0.9516601934353819, 'rating': 7.6, 'price': 1, 'distance': 565, 'fsq_id': '4b182f85f964a52085ce23e3'}, {'latitude': 49.279764, 'longitude': -123.110154, 'station_id': '6d42fa40360f9a6b2bf641c7b8bb2862', 'name': 'Six Acres', 'popularity': 0.9387708476125199, 'rating': 8.1, 'price': 1, 'distance': 582, 'fsq_id': '4aa7329cf964a5203f4c20e3'}, {'latitude': 49.260599, 'longitude': -123.113504, 'station_id': '66f873d641d448bd1572ab086665a458', 'name': 'Sing Sing Beer Bar', 'popularity': 0.9871296930926814, 'rating': 7.4, 'price': None, 'distance': 921, 'fsq_id': '5c6f3cc2c0363

In [ ]:
## Convert the 'all_results' list of dictionaries to a DataFrame

In [113]:
df_foursquare = pd.DataFrame(all_results)
df_foursquare

,latitude,longitude,station_id,name,popularity,rating,price,distance,fsq_id
0,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Mahony & Sons,0.995269,7.4,NaN,811,53eee0b9498e381a6c7eaa98
1,49.274566,-123.121817,32603a87cfca71d0f7dfa3513bad69d5,Localcoin Bitcoin ATM - Red Card Sports Bar + ...,0.951660,7.6,1.0,565,4b182f85f964a52085ce23e3
2,49.279764,-123.110154,6d42fa40360f9a6b2bf641c7b8bb2862,Six Acres,0.938771,8.1,1.0,582,4aa7329cf964a5203f4c20e3
3,49.260599,-123.113504,66f873d641d448bd1572ab086665a458,Sing Sing Beer Bar,0.987130,7.4,NaN,921,5c6f3cc2c03635002c74c211
4,49.264215,-123.117772,485d4d24c803cfde829ab89699fed833,Victoria's Health and Organic Bar,0.090797,NaN,1.0,131,58f13221419a9e1e24489f12
...,...,...,...,...,...,...,...,...,...
240,49.280977,-123.035969,a74744ce4bb7ea2aa9f406ac8bff95d8,The Rustic Llama Cafe,0.873353,NaN,1.0,870,4e9f85bf30f854cd9bf64a29
241,49.270783,-123.141564,5699b40126e10c2f68eefc3eb18ff3a1,Lucky Taco,0.869412,6.7,1.0,981,57b26a56cd104357811923e4
242,49.264019,-123.209176,34fd37d12eb989b49518ef53941ee3ff,Varsity Barbers,0.751961,NaN,NaN,282,509ad61591d477646f22cb2f
243,49.265800,-123.205960,b7dd37a7dd668d6d10024b7f18acc438,The Good Stuff,0.769515,NaN,NaN,335,5978d136851de569bc454a8a


In [114]:
## SAVE A BACKUP of MY PRECIOUS JSON FILE

import json

with open('../data/FourSquare_APIresult_data.json', 'w') as f:
    f.write(json.dumps(json.loads(df_foursquare.to_json(orient='records')), indent=4))


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [20]:
import YELP as creds
import requests
import os
API_KEY=creds.YELP_API
print(API_KEY) ## Confirmed - API Key ok!! 

def yelp_API_call(latitude, longitude, radius):
    parameters = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "categories":"bars%2Crestaurants"
    }
    url = "https://api.yelp.com/v3/businesses/search"

    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {API_KEY}" 
    }

    response = requests.get(url, params=parameters, headers=headers)
    print(response.text)

yelp_API_call(49.262487,-123.114397,1000)

epLDm9--bpzcKYMtzI8Nn82QafY6Nwed1dH4wo8X1FZ_yZcdgoP8V1hhqQcuGkrNMhl1SiZW2A75shK2OHmNjY3FcyqvJXc_aLYFYiiR8Sk9OuWvtgfnoR6_qu4NZXYx
{"businesses": [{"id": "6iOAgzJ0DRZNSKA3FSrrOg", "alias": "la-taqueria-pinche-taco-shop-vancouver", "name": "La Taqueria Pinche Taco Shop", "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/fjhIj3XKuQ4mquD4Mg8OoQ/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/la-taqueria-pinche-taco-shop-vancouver?adjust_creative=v02fG6FieACbA2bIWEgDDA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=v02fG6FieACbA2bIWEgDDA", "review_count": 681, "categories": [{"alias": "mexican", "title": "Mexican"}], "rating": 4.0, "coordinates": {"latitude": 49.263559, "longitude": -123.112736}, "transactions": [], "price": "$$", "location": {"address1": "2450 Yukon Street", "address2": "", "address3": "", "city": "Vancouver", "zip_code": "V5Z 3V6", "country": "CA", "state": "BC", "display_address": ["2450 Yukon Street", "Vancouver, BC V5Z 3V6", "Cana

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating